In [1]:
import pandas as pd
import re
import string

In [5]:
real_news = pd.read_csv("News_Dataset/True.csv")
fake_news = pd.read_csv("News_Dataset/Fake.csv")
real_news2 = pd.read_csv("addtional_training_data/True2")
fake_news2 = pd.read_csv("addtional_training_data/Fake2")

FileNotFoundError: [Errno 2] No such file or directory: 'addtional_training_data/True2'

In [3]:
real_news.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [4]:
fake_news.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [5]:
real_news["label"] = 1
fake_news["label"] = 0

In [6]:
real_news.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [7]:
real_news_model = real_news[["title", "label"]]
fake_news_model = fake_news[["title", "label"]]

In [8]:
complete_df = pd.concat([real_news_model, fake_news_model], axis=0)
complete_df = complete_df.sample(frac=1, random_state=42).reset_index(drop=True)
complete_df.head()

,title,label
0,BREAKING: GOP Chairman Grassley Has Had Enoug...,0
1,Failed GOP Candidates Remembered In Hilarious...,0
2,Mike Pence’s New DC Neighbors Are HILARIOUSLY...,0
3,California AG pledges to defend birth control ...,1
4,AZ RANCHERS Living On US-Mexico Border Destroy...,0


In [10]:
complete_df = complete_df.dropna()

In [11]:
complete_df.isnull().sum()

title    0
label    0
dtype: int64

In [13]:
complete_df = complete_df.drop_duplicates(subset=["title"], keep="first")
complete_df.shape

(38729, 2)

In [16]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans("","", string.punctuation))
    text = text.strip()
    return text

complete_df["title"] = complete_df["title"].apply(clean_text)

complete_df.head()

,title,label
0,breaking gop chairman grassley has had enough ...,0
1,failed gop candidates remembered in hilarious ...,0
2,mike pence’s new dc neighbors are hilariously ...,0
3,california ag pledges to defend birth control ...,1
4,az ranchers living on usmexico border destroy ...,0


In [17]:
complete_df.to_csv("cleaned_news_headlines.csv", index=False)